In [6]:
# Import necessary libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_experimental.text_splitter import SemanticChunker
from langchain.schema import Document

from pathlib import Path
import re

# Define document paths
doc_paths = [
    "docs/가스계통_운영규정.pdf",
    "docs/여비규정.pdf",
    "docs/취업규칙.pdf",
]

# Load documents
docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    print("Loading document:", doc_file)
    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
            for page in loader.load_and_split():
                # Create a new Document with page number in metadata
                doc = Document(
                    page_content=page.page_content,
                    metadata={
                        "source": doc_file,
                        "page": page.metadata["page"]
                    }
                )
                docs.append(doc)
    except Exception as e:
        print(f"Error loading document {doc_file}: {e}")

# Split documents
text_splitter = SemanticChunker(
    OpenAIEmbeddings()
)

document_chunks = text_splitter.split_documents(docs)

# Create vector store
vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

# Create retrievers
bm25_retriever = BM25Retriever.from_documents(document_chunks)
bm25_retriever.k = 3

chroma_retriever = vector_db.as_retriever(search_kwargs={'k': 3})

ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.5, 0.5]
)

# Create LLM
llm = ChatOpenAI(temperature=0)

# Create MultiQueryRetriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=ensemble_retriever,
    llm=llm
)

def _get_context_retriever_chain(vector_db, ensemble_retriever, llm):
    multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever=ensemble_retriever,
        llm=llm
    )

    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, ensemble_retriever, prompt)

    return retriever_chain

def get_conversational_rag_chain(llm, retriever):
    retriever_chain = _get_context_retriever_chain(vector_db, retriever, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are an assistant designed specifically for answering queries based on company regulations. Always respond strictly according to the company's internal regulations, ensuring your answers are aligned with these rules. 
        When providing an answer, first cite the most relevant regulation in detail, including chapter and section numbers if applicable. If multiple regulations apply, list all relevant ones before giving your response. 
        Your goal is to provide the user with clear guidance based on the regulations, so be as specific as possible with the details of the rules and regulations before proceeding with the final answer.
        If no regulation directly applies, inform the user and give your best guidance based on your knowledge of the company's practices.
        
        After your explanation, provide the exact quotes from the relevant regulations under a "Source Regulations:" section. Format each quote as follows:
        [Document Name] Chapter X, Section Y (Page Z): "Exact quote from the regulation"

        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

# Create streaming LLM
llm_stream_openai = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    streaming=True,
)

# Define messages
messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "일이 너무 많아서 연차휴가를 할당된 만큼 다 쓰지 못할거 같아. 그럼 남는 연차휴가는 어떻게 되지? 남는 연차 휴가에 대해 돈으로 받을수 있어?"},
]

# Convert dict messages to HumanMessage and AIMessage objects
langchain_messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

# Create conversational RAG chain
conversation_rag_chain = get_conversational_rag_chain(llm_stream_openai, multi_query_retriever)

# Process response
def process_response(response):
    source_regex = r"Source Regulations:(.*?)(?=\n\n|\Z)"
    source_match = re.search(source_regex, response, re.DOTALL)
    
    if source_match:
        sources = source_match.group(1).strip().split('\n')
        for source in sources:
            match = re.match(r"\[(.*?)\](.*?)\(Page (\d+)\): \"(.*)\"", source)
            if match:
                doc_name, section, page, quote = match.groups()
                print(f"\nReference from {doc_name}:")
                print(f"Section: {section.strip()}")
                print(f"Page: {page}")
                print(f"Quote: {quote}")
                print("------------------------")

# Generate and process response
response_message = "*(RAG Response)*\n"
last_user_message = messages[-1]["content"]

for chunk in conversation_rag_chain.pick("answer").stream({"messages": langchain_messages[:-1], "input": last_user_message}):
    response_message += chunk
    print(chunk, end="", flush=True)

print("\n\nDetailed Source Information:")
process_response(response_message)

# Append the assistant's response to the messages list
messages.append({"role": "assistant", "content": response_message})

# Print final messages for verification
print("\nFinal Messages:")
for msg in messages:
    print(f"{msg['role']}: {msg['content'][:50]}...")  # Print first 50 characters

Loading document: docs/가스계통_운영규정.pdf
Loading document: docs/여비규정.pdf
Loading document: docs/취업규칙.pdf
제29조(연차유급휴가)와 관련된 규정을 살펴보면, 연차휴가를 다 사용하지 못한 경우에 대한 규정이 있습니다.

1. 제29조 제5항에 따르면, 연차유급휴가 중 12일의 범위 내에서 사용촉진조치를 시행하며, 그 미사용 휴가는 연간 10일 한도로 5년간 저축하여 사용할 수 있습니다. 그러나 5년 이내 또는 퇴직 시까지 사용하지 않은 저축 연차휴가는 자동소멸됩니다.

2. 제29조 제6항에 따르면, 회사의 형편에 따라 연차휴가를 사용할 수 없거나 적치하지 않은 경우에는 보수규정이 정하는 바에 따라 수당을 지급할 수 있습니다.

따라서, 연차휴가를 다 사용하지 못한 경우, 일부는 저축하여 사용할 수 있으며, 회사의 사정에 따라 수당으로 받을 수도 있습니다.

Source Regulations:
- [Document Name] 제29조(연차유급휴가) 제5항: "연차휴가는 직원의 자유의사에 따라 적치하여 계산기간 만료 익일부터 1년 이내에 사용한다. 단, 공사가 제29조 제1항, 제3항 및 제4항에 따른 연차유급휴가 중 12일의 범위 내에 근로기준법 제61조에 따라 사용촉진조치를 시행하며 그 미사용 휴가는 연간 10일 한도로 5년간 저축 사용 할 수 있으나 5년 이내 또는 퇴직 시까지 사용하지 아니한 저축 연차휴가는 자동소멸된다."
- [Document Name] 제29조(연차유급휴가) 제6항: "공사의 형편에 따라 연차휴가를 사용할 수 없거나 적치하지 아니한 때에는 보수규정이 정하는 바에 따라 수당을 지급한다."

Detailed Source Information:

Final Messages:
user: Hi...
assistant: Hi there! How can I assist you today?...
user: 일이 너무 많아서 연차휴가를 할당된 만큼 다 쓰지 못할거 같아. 그럼 남는